In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import folium
import branca
import panama_papers_aux

###### Versions

In [2]:
print('ipywidgets:', widgets.__version__)
print('folium:', folium.__version__)

ipywidgets: 7.0.0
folium: 0.5.0


# Simple queries

### Reading clean data

In [3]:
entities = pd.read_csv("./data/data_clean_csv/entities_clean.csv", dtype = 'object')
intermediaries = pd.read_csv("./data/data_clean_csv/intermediaries_clean.csv", dtype = 'object')
officers = pd.read_csv("./data/data_clean_csv/officers_clean.csv", dtype = 'object')
addresses = pd.read_csv("./data/data_clean_csv/addresses_clean.csv", dtype = 'object')
all_edges = pd.read_csv("./data/data_clean_csv/all_edges_clean.csv", dtype = 'object')

### DataFrame dictionary creation

We now create a dictionary that contains all the DataFrames:

In [4]:
df_dictionary = {'Entity': entities, 'Intermediary': intermediaries, 'Officer': officers, 'Address': addresses,
             'all_edges': all_edges}

### List of available countries

`available_countries` is a list containing all the countries that can be used as a filter for the query.

In [5]:
available_countries = [x for x in officers['countries'].unique() if ';' not in x]
available_countries.sort()
available_countries = ['All'] + available_countries
len(available_countries)

208

### Query function definition

Query function that can be applied to `entities`, `ìntermediaries`, `officers` and `addresses`. The query can be filtered thanks by selecting a particular country.

In [6]:
def search_by_name(queried_name, dictionary, df_name, country):
    df = dictionary[df_name]
    df_copy = df.copy()
    if (country != 'All'):
        df_copy = df_copy[df_copy['countries'].str.contains(country, case = False)]        
    matched_df_copy = df_copy[df_copy['name'].str.contains(queried_name, case = False)]
    matched_name_serie = matched_df_copy['name']

    if (matched_name_serie.shape[0] > 100):
        matched_name_serie = matched_name_serie.iloc[:100]
        matched_df_copy = matched_df_copy.iloc[:100]

    return matched_df_copy

### GUI to get the relationships of a particular item

The `get_relationships` function is meant to find all the relationships of a particular item of the database, represented as DataFrame line that is passed as input (`df_queried_item`). It outputs a DataFrame showing all the in and out-relationships involving the queried item of the database.
This function is then used through a GUI interface at the end of this notebook.

In [7]:
panama_papers_aux.Item.readCapitalCoordinates()

In [8]:
options_type_dic = {'Entities': 'entities', 'Intermediaries': 'intermediaries', 'Officers': 'officers'}

option_list = ['Entity', 'Intermediary', 'Officer']
widgetDFName = widgets.Dropdown(description = 'Type', options = option_list)
widgetQueriedName = widgets.Text(description = 'Name', value = 'Type name here')
widgetCountry = widgets.Dropdown(description = 'Country', options = available_countries)
# 1st widget
first_widget = interactive(search_by_name, queried_name = widgetQueriedName, dictionary = fixed(df_dictionary),\
                df_name = widgetDFName, country = widgetCountry)

# Button 1
button1 = widgets.Button(description="Apply Filter")
def show_results(b):
    widgetResult.options = first_widget.result['name'].values.tolist()
button1.on_click(show_results)

# 2nd widget
def name_selection(selected_name):
    matched_df = first_widget.result
    return matched_df[matched_df['name'] == selected_name]
widgetResult = widgets.Dropdown(description = 'Name options')
second_widget = interactive(name_selection, selected_name = widgetResult)

# Button 2
def print_selected_result(b):
    if (second_widget.result.shape[0] != 0):
        # Instantiating an Item object corresponding to the query
        # Then instantiating a NetworkItem object from the previous Item object
        type_ = widgetDFName.value
        queried_item = panama_papers_aux.Item.fromSingleLineDF(second_widget.result, type_)
        queried_item_network = panama_papers_aux.ItemNetwork(queried_item, df_dictionary)
        
        # get ItemNetwork DataFrame and print it
        network_df = queried_item_network.getDF()
        display(network_df)
        
        # get FoliumMap of connections and print it
        connection_map = queried_item_network.getMap()
        display(connection_map)
        button2.close()
        #button3.open()

button2 = widgets.Button(description="Display relationships")
button2.on_click(print_selected_result)

handle = display(first_widget, button1, second_widget, button2)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

### Print the relationships on a folium map

The `print_relationships_on_map` is a function that calls 

In [9]:
def relationships_on_map(relationships_output_df):
    pass

In [10]:
def nodes_on_map(original_map, relationships_output_df):
    # firstly we have to get the nodes to print
    # To do so we maintain a list with the node id that have to be printed 
    #    and we go through the relationships_output_df
    
    in_nodes_list = []
    
    node_list = [];
    for df_line in relationships_output_df:
        pass
    
    

In [11]:
#inputs
original_map = folium.Map(location=[0, 0], zoom_start =2, min_zoom=2)
relationships_output_df = get_relationships(second_widget.result)

noded_map = nodes_on_map(original_map, relationships_output_df)

NameError: name 'get_relationships' is not defined

In [11]:
fol_map = folium.Map(location=[0, 0], zoom_start =2, min_zoom=2)


cm = branca.colormap.LinearColormap(['green', 'yellow', 'red'], vmin=0, vmax=50)
print(cm(25))
cm
feat = folium.map.FeatureGroup()
feat.add_child(
        folium.features.CircleMarker(
            [48.8666666666667, 2.333333],
            radius=8,
            color='#3388ff',
            fill_color=cm(25),
            fill = True,
            popup = "Hello",
            fill_opacity=0.6)
        )

fol_map.add_child(feat)
fol_map

#ffff00


In [ ]:
country_df = pd.read_csv('./res/country-capitals.csv')
country_df[country_df['CountryName'] == 'United States']['CapitalLatitude'].values[0]